In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
with open('X.npy', 'rb') as f:
    X = np.load(f)

with open('y.npy', 'rb') as f:
    y = np.load(f)

FileNotFoundError: ignored

In [ ]:
#K-fold cross validation : Data가 작을 때 validation accuracy를 믿고자 하면

#초기는, X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, randoma_state =42)
#처럼, test_size를 0.1로 10% 잘라서 테스트를 하였다
X.shape, y.shape

((1460, 60), (1460, 1))

In [ ]:
x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)
scaled_X = x_min_max_scaler.transform(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)
scaled_y = y_min_max_scaler.transform(y)

In [ ]:
# K-fold cross validation
K = 10
kf = KFold(n_splits=K)

In [ ]:
rmses = []
for train_index, test_index in kf.split(scaled_X):
  scaled_X_train, scaled_X_test = scaled_X[train_index], scaled_X[test_index]
  scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
  y_test = y[test_index]

  # training
  model = keras.Sequential(
      [
          keras.Input(shape=scaled_X_train.shape[-1]),
          layers.Dense(96, activation='relu'),
          layers.Dense(48, activation='relu'),
          layers.Dense(1)
      ]
  )

  opt = keras.optimizers.Adam(learning_rate=0.005)
  model.compile(loss="mse", optimizer=opt)

  early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
  model.fit(scaled_X_train, scaled_y_train, 
            batch_size=2, epochs=150, 
            callbacks=[early_stopping_callback], validation_split=0.05, verbose='auto')

  # evaluation
  pred = model.predict(scaled_X_test).reshape((-1, 1))
  pred = y_min_max_scaler.inverse_transform(pred)
  rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))

  print(rmse)
  print("---------------------")
  
  rmses.append(rmse)
  break

print("average rmse:", np.mean(rmses))

Epoch 1/150
624/624 [==============================] - 2s 2ms/step - loss: 0.0069 - val_loss: 0.0057
Epoch 2/150
624/624 [==============================] - 1s 1ms/step - loss: 0.0043 - val_loss: 0.0030
Epoch 3/150
624/624 [==============================] - 1s 1ms/step - loss: 0.0035 - val_loss: 0.0030
Epoch 4/150
624/624 [==============================] - 1s 1ms/step - loss: 0.0035 - val_loss: 0.0017
Epoch 5/150
624/624 [==============================] - 1s 2ms/step - loss: 0.0034 - val_loss: 0.0018
Epoch 6/150
624/624 [==============================] - 1s 2ms/step - loss: 0.0028 - val_loss: 0.0022
Epoch 7/150
624/624 [==============================] - 1s 2ms/step - loss: 0.0025 - val_loss: 0.0013
Epoch 8/150
624/624 [==============================] - 1s 1ms/step - loss: 0.0030 - val_loss: 0.0015
Epoch 9/150
624/624 [==============================] - 1s 1ms/step - loss: 0.0020 - val_loss: 0.0015
Epoch 10/150
624/624 [==============================] - 1s 1ms/step - loss: 0.0019 - val_lo

# Hyperparameters

- batch size
- learning rate
- number of layers
- feature sizes
- activation functions
- optimizers

# Grid search

In [ ]:
# batch_size, learning_rate 

In [ ]:
batch_sizes = np.arange(5, 10, 2)

In [ ]:
batch_sizes

array([5, 7, 9])

In [ ]:
learning_rates = [0.005, 0.01, 0.02]

In [ ]:
for batch_size in batch_sizes:
  for learning_rate in learning_rates:
    print("batch_size:", batch_size, "learning_rate:", learning_rate)

batch_size: 5 learning_rate: 0.005
batch_size: 5 learning_rate: 0.01
batch_size: 5 learning_rate: 0.02
batch_size: 7 learning_rate: 0.005
batch_size: 7 learning_rate: 0.01
batch_size: 7 learning_rate: 0.02
batch_size: 9 learning_rate: 0.005
batch_size: 9 learning_rate: 0.01
batch_size: 9 learning_rate: 0.02


In [ ]:
results = []

for batch_size in batch_sizes:
  for learning_rate in learning_rates:
    print(batch_size, learning_rate)
    for train_index, test_index in kf.split(scaled_X):
      scaled_X_train, scaled_X_test = scaled_X[train_index], scaled_X[test_index]
      scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
      y_test = y[test_index]

      # training
      model = keras.Sequential(
          [
              layers.InputLayer(input_shape=scaled_X_train.shape[-1]),
              layers.Dense(96, activation='relu'), # (0.1439302, 0.123, 0.999) --> (23, 0, 255)
              layers.Dense(48, activation='relu'),
              layers.Dense(1)
          ]
      )

      #model.compile(loss="mse", optimizer="adam")

      opt = keras.optimizers.Adam(learning_rate=0.005)
      model.compile(loss="mse", optimizer=opt)

      early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
      model.fit(scaled_X_train, scaled_y_train, 
                batch_size=1, epochs=150, 
                callbacks=[early_stopping_callback], validation_split=0.05, verbose='auto')

      # evaluation
      pred = model.predict(scaled_X_test).reshape((-1, 1))
      pred = y_min_max_scaler.inverse_transform(pred)
      rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))

      print(rmse)
      print("---------------------")

      result = {}
      result['batch_size'] = batch_size
      result['learning_rate'] = learning_rate
      result['rmse'] = rmse

      results.append(result)
      break

    
  break

In [ ]:
results

[{'batch_size': 5, 'learning_rate': 0.005, 'rmse': 26624.27111252062},
 {'batch_size': 5, 'learning_rate': 0.01, 'rmse': 31153.655918156775},
 {'batch_size': 5, 'learning_rate': 0.02, 'rmse': 24148.90108388349}]

# Random Search

In [ ]:
import random

In [ ]:
batch_sizes = random.sample(range(1, 11), 5)

In [ ]:
batch_sizes

[6, 2, 10, 7, 1]

In [ ]:
learning_rates = np.random.uniform(low=0.005, high=0.1, size=(4,))

NameError: ignored

In [ ]:
learning_rates

# Model Quantization

In [1]:
!pip install -q tensorflow-model-optimization

     |████████████████████████████████| 213 kB 7.6 MB/s 


In [2]:
import tensorflow_model_optimization as tfmot

In [3]:
quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
# 플로팅과 int 레이어 모두를, q_aware_model 안에 생성
# 즉, int8용 데이터가 들어갈 공간을 생성한 것.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
# 나머진 케라스 모델과 동일
opt = keras.optimizers.Adam(learning_rate=0.005)
q_aware_model.compile(optimizer=opt,
                      loss="mse")

q_aware_model.summary()

NameError: ignored

In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
q_aware_model.fit(scaled_X_train, scaled_y_train, 
                  batch_size=2, epochs=150, 
                  callbacks=[early_stopping_callback], validation_split=0.05, verbose='auto')

## Convert into quantized model (int8)

In [4]:
#lite버전을 통해, int용으로 경량화시킨 것
# int8에 해당하는 q_aware_model을 convert함.
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

NameError: ignored

In [5]:
#이 int8버전을 inference하고자함.
#.predict같은 것이 아니라, 
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

NameError: ignored

## Inference quantized model

In [6]:
#input, output index 만들고
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

NameError: ignored

In [7]:
scaled_X.shape

NameError: ignored

In [ ]:
float32_X = scaled_X.astype(np.float32)

In [ ]:
float32_X.shape

In [ ]:
# 앞에서 load(컨버전)한 interpreter에 input을 넣게 됨.
# 예측하고자하는 것 중 [0]번쨰 zip을 분석. input index에 텐서를 넣음
interpreter.set_tensor(input_index, [float32_X[0]])

In [ ]:
# Run inference.
interpreter.invoke()

In [ ]:
#ouput에 해당하는 idx를 뽑아냄.
output = interpreter.tensor(output_index)

In [ ]:
output()

In [ ]:
#scale 원상복구
y_min_max_scaler.inverse_transform(output())

# Check model size

In [8]:
import os, tempfile

In [9]:
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

NameError: ignored

In [ ]:
#실제로, 모델 사이즈를 비교하면
#float모델보다 int모델이 1/4수준인 것을 볼 수 있다.
#데이터 정확도 차이는 별로 안 남에도